In [ ]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.1/240.1 kB 21.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-00-bf16881c8728


In [3]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [4]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [5]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.bq_embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [6]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [7]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [8]:
QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'More and more Americans are joining the internet\'s fast lane, according to official figures.\n\nThe number of people and business connected to broadband jumped by 38% in a year, said the US Federal Communications Commission (FCC). In a report, it said there were more than 32 million broadband connections by the end of June 2004. But the US is still behind compared to other nations, ranked 13th in the world by a UN telecoms body.\n\nDuring his 2004 re-election campaign, President George W Bush pledge to ensure that affordable high-speed net access would be available to all Americans by 2007.\n\nAccording to the report by the FCC, broadband is becoming increasingly popular, with people using it for research and shopping, as well as downloading music and watching video. The total number of people and businesses on broadband rose by to 32.5 million in the year ending June 2004, compared to 23.5 million in June 2003. Whereas in the UK, most people hook up to broadband via Asymmetric Digit

' . . . '

'User Question:'

'Tell me about the US Economy'

'--------------------------------'

'Gemini Answer:'

'The US economy is the largest in the world, with a GDP of over $21 trillion in 2023. It is a mixed economy, with a significant private sector and a smaller public sector. The US is a global leader in many industries, including technology, finance, and agriculture. The country has a highly skilled workforce and a strong infrastructure. However, the US economy also faces some challenges, such as income inequality and a large national debt.\n\nHere are some specific facts about the US economy:\n\n* **GDP:** $21.43 trillion (2023 est.)\n* **GDP per capita:** $63,413 (2023 est.)\n* **Unemployment rate:** 3.7% (October 2023)\n* **Inflation rate:** 7.7% (October 2023)\n* **Major industries:** technology, finance, agriculture, healthcare, manufacturing\n* **Major trading partners:** China, Mexico, Canada, Japan, Germany\n\nThe US economy is expected to grow by 2.2% in 2023 and 2.0% in 2024. The Federal Reserve is expected to continue raising interest rates in 2023 to combat inflation. The US 

In [ ]:
QUESTION = "What it a good recipe for bouillabaisse"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'Voting is under way for the annual Bloggies which recognise the best web blogs - online spaces where people publish their thoughts - of the year.\n\nNominations were announced on Sunday, but traffic to the official site was so heavy that the website was temporarily closed because of too many visitors. Weblogs have been nominated in 30 categories, from the top regional blog, to the best-kept-secret blog. Blogs had a huge year, with a top US dictionary naming "blog" word of 2004. Technorati, a blog search engine, tracks about six million blogs and says that more than 12,000 are added daily. A blog is created every 5.8 seconds, according to US research think-tank Pew Internet and American Life, but less than 40% of the total are updated at least once every two months.\n\nNikolai Nolan, who has run the Bloggies for the past five years, told the BBC News website he was not too surprised by the amount of voters who crowded the site. "The awards always get a lot of traffic; this was just my 

' . . . '

'User Question:'